The Turán number of $K_4^-$
=================================

In [1]:
from sage.algebras.flag_algebras import *

# This is some hack to create the theory for 3-graphs without C5- and K4-
# up to size 7. It is easier to make them as extensions of 6 sized structures
# so this code does that.

def check_containment(smalls, larges):
    """
    Helper function to check is any of the smalls appears in each of the larges.

    INPUT:
    smalls - list of flags, must be from a theory with edges relation
    larges - list of flags, also must be from a theory with edges relation

    OUTPUT:
    list of booleans, i-th element represents if i-th large flag is free from all smalls
    """
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in larges]
    res = []
    for ll in lis:
        good = True
        for ss in sis:
            for _ in ll.isomorphic_substructures_iterator(ss):
                good = False
                break
            if not good:
                break
        res.append(good)
    return res

# Reset three graphs, so nothing is excluded
TG = ThreeGraphTheory
# k4 and k4m (the two induced structures with size 4 excluded)
k4 = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]])
k4m = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])
excls = [k4,k4m]
TG.exclude(excls)

# check the list of flags with size 5 and 6
fl5 = TG.generate_flags(5)
fl6 = TG.generate_flags(6)

# quick 3-graph identifier code. This will be the identifier for
# the theory of C5- free 3-graphs (any identifier working for 3-graphs can work here)
def _identify_hypergraph(n, ftype_points, edges):
    g = Graph([list(range(n+len(edges))), [(i+n,x) for i,b in enumerate(edges) for x in b]], 
              format='vertices_and_edges')
    ftype_union = [jj for ff in ftype_points for jj in ff]
    partt = list(ftype_points) + \
            [[ii for ii in range(n) if ii not in ftype_union]] + \
            [list(range(n,n+len(edges)))]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    return (n, tuple([len(xx) for xx in ftype_points]), blocks)

# generator code. It should really just return TG, but for size 7 that takes too long
# so this hack just returns TG for size up to 6, and for 7 it generates all flags
# with this extension technique
def _gen(n):
    if n<=4:
        for xx in TG.generate_flags(n):
            yield xx.blocks()
    elif n==5:
        for xx in fl5:
            yield xx.blocks()
    elif n==6:
        for xx in fl6:
            yield xx.blocks()
    elif n==7:
        import itertools
        from tqdm import tqdm
        fl7_m = [[] for ii in range(35+1)]
        subs = list(itertools.combinations(range(6), int(2)))
        for xx in tqdm(fl6):
            xb = xx.blocks()['edges']
            for ii in range(15+1):
                for pps in itertools.combinations(subs, int(ii)):
                    xbp = [[pp[0], pp[1], 6] for pp in pps] + xb
                    flxp = TG(7, edges=xbp)
                    en = len(xbp)
                    if flxp not in fl7_m[en]:
                        if check_containment(excls, [flxp])[0]:
                            fl7_m[en].append(flxp)
        fl7 = [yy for xx in fl7_m for yy in xx]
        for xx in fl7:
            yield xx.blocks()
    else:
        #for n>=8 just return an empty list, this will not be called so doesn't 
        #really matter
        return []

# Create the theory based on this generator and identifier
TGp = CombinatorialTheory("NoK4m", _gen, _identify_hypergraph, edges=3)

# for sanity check, print the number of structures with size 5, 6, 7
# should be 11 106 8157
print(len(TGp.generate_flags(5)), len(TGp.generate_flags(6)), len(TGp.generate_flags(7)))

100%|█████████████████████████████████████████| 106/106 [11:05<00:00,  6.28s/it]

11 106 8157


In [9]:
tp = TGp(3, ftype=[0, 1, 2], edges=[[0, 1, 2]])
lstp = TGp.generate_flags(6, tp)
smtp = TGp.generate_flags(4, tp)
smtp[2].subflag([3])

Flag on 4 points, ftype from [0, 2, 1] with edges=[[0, 1, 2], [0, 1, 3]]

In [16]:
tp = TGp(3, ftype=[0, 1, 2], edges=[[0, 1, 2]])
lstp = TGp.generate_flags(6, tp)
smtp = TGp.generate_flags(4, tp)
targ_flag = -1
for xx in lstp:
    uniques = []
    stars = 0
    error = False
    for ii in range(3):
        subf = xx.subflag([xx.not_ftype_points()[ii]])
        try:
            ind = smtp.index(subf)
        except:
            print(smtp)
            print(subf)
            error = True
            break
        if ind==0:
            stars += 1
        else:
            if ind not in uniques:
                uniques.append(ind)
    if error:
        break
    if len(uniques)==0:
        targ_flag += xx/9
    elif len(uniques)==1:
        targ_flag += xx*(3**(-stars))
targ_flag += 1
targ_flag

Flag Algebra Element over Rational Field
1/9 - Flag on 6 points, ftype from [0, 1, 2] with edges=[[0, 1, 2]]
1/9 - Flag on 6 points, ftype from [0, 1, 2] with edges=[[0, 1, 2], [0, 1, 3]]
1/9 - Flag on 6 points, ftype from [0, 2, 1] with edges=[[0, 1, 2], [0, 1, 3]]
1/9 - Flag on 6 points, ftype from [2, 0, 1] with edges=[[0, 1, 2], [0, 1, 3]]
1/9 - Flag on 6 points, ftype from [0, 1, 2] with edges=[[0, 1, 2], [0, 3, 4]]
1/9 - Flag on 6 points, ftype from [1, 0, 2] with edges=[[0, 1, 2], [0, 3, 4]]
1/9 - Flag on 6 points, ftype from [1, 2, 0] with edges=[[0, 1, 2], [0, 3, 4]]
1/9 - Flag on 6 points, ftype from [0, 1, 2] with edges=[[0, 1, 2], [3, 4, 5]]
1/3 - Flag on 6 points, ftype from [0, 1, 2] with edges=[[0, 1, 2], [0, 1, 3], [0, 1, 4]]
1/3 - Flag on 6 points, ftype from [0, 2, 1] with edges=[[0, 1, 2], [0, 1, 3], [0, 1, 4]]
1/3 - Flag on 6 points, ftype from [2, 0, 1] with edges=[[0, 1, 2], [0, 1, 3], [0, 1, 4]]
1/9 - Flag on 6 points, ftype from [0, 1, 3] with edges=[[0, 1, 2], 

In [17]:
f = TGp(6, edges=[[1,2,3], [2,3,4], [3,4,5], [4,5,1], [5,1,2], [1,3,0], [2,4,0], [3,5,0], [4,1,0], [5,2,0]])
degree = TGp(3, edges=[ [0,1,2] ], ftype=[0])
p2f4 = TGp.generate_flags(4, TGp(2, ftype=[0, 1]))
degree_difference = p2f4[2]-p2f4[3]+p2f4[5]-p2f4[6]
min_F = TGp.optimize_problem(f, 6, maximize=False, positives=[degree-2/7, degree_difference, -degree_difference])

#around 23Gb memory required for size 7
#block sizes are [5, 95, 47, 388, 262, 191, 173, 148, 135, 124, 101, 95, 94, 72, -8157, -26]

Base flags generated, their number is 106
The relevant ftypes are constructed, their number is 4
Block sizes before symmetric/asymmetric change is applied: [8, 41, 26, 18]


Done with mult table for Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]]: : 4it [00:06,  1.58s/it]


Tables finished


Done with positivity constraint 2: 100%|██████████| 3/3 [00:02<00:00,  1.01it/s]


Constraints finished
Running sdp without construction. Used block sizes are [6, 2, 7, 34, 8, 18, 9, 9, -106, -23]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.41e-01 Pobj: -2.7509910e+01 Ad: 5.58e-01 Dobj:  3.5603859e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -3.3435032e+01 Ad: 8.72e-01 Dobj:  3.6761685e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -3.4024832e+01 Ad: 9.26e-01 Dobj:  5.0691002e-02 
Iter:  4 Ap: 1.00e+00 Pobj: -3.3545708e+01 Ad: 5.92e-01 Dobj:  2.2567706e-02 
Iter:  5 Ap: 4.60e-01 Pobj: -2.9798883e+01 Ad: 4.92e-01 Dobj:  1.3808841e-02 
Iter:  6 Ap: 4.75e-01 Pobj: -2.4229224e+01 Ad: 5.36e-01 Dobj:  1.3760916e-02 
Iter:  7 Ap: 8.27e-01 Pobj: -1.8150144e+01 Ad: 5.77e-01 Dobj:  1.0010971e-02 
Iter:  8 Ap: 1.00e+00 Pobj: -1.2247915e+01 Ad: 5.93e-01 Dobj:  1.0575398e-02 
Iter:  9 Ap: 1.00e+00 Pobj: -9.7620314e+00 Ad: 5.10e-01 Dobj:  1.1318036e-02 
Iter: 10 Ap: 8.17e-01 Pobj: -7.3754172e+00 Ad: 6.93e-01 Dobj:  1.0895022e-02 
Iter: 11 Ap: 4.92

In [18]:
max_F3 = TGp.optimize_problem(targ_flag, 6, maximize=True, positives=[degree-2/7, degree_difference, -degree_difference])

Base flags generated, their number is 106
The relevant ftypes are constructed, their number is 4
Block sizes before symmetric/asymmetric change is applied: [8, 41, 26, 18]


Done with mult table for Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]]: : 4it [00:00, 5279.17it/s]


Tables finished


Done with positivity constraint 2: 100%|█████████| 3/3 [00:00<00:00, 237.25it/s]


Constraints finished
Running sdp without construction. Used block sizes are [6, 2, 7, 34, 8, 18, 9, 9, -106, -23]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.14e-01 Pobj: -6.2651594e+01 Ad: 2.98e-01 Dobj: -5.7239638e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -1.0132024e+02 Ad: 6.81e-01 Dobj: -1.3002687e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.0569506e+02 Ad: 9.07e-01 Dobj: -1.0193505e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0508095e+02 Ad: 7.32e-01 Dobj: -8.6586085e-02 
Iter:  5 Ap: 6.38e-01 Pobj: -7.1261497e+01 Ad: 4.66e-01 Dobj: -1.0138286e-01 
Iter:  6 Ap: 6.46e-01 Pobj: -6.2242595e+01 Ad: 7.16e-01 Dobj: -1.1565089e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -3.9620836e+01 Ad: 6.12e-01 Dobj: -1.1740411e-01 
Iter:  8 Ap: 9.68e-01 Pobj: -2.5292390e+01 Ad: 5.48e-01 Dobj: -1.1437758e-01 
Iter:  9 Ap: 1.00e+00 Pobj: -2.2928065e+01 Ad: 6.42e-01 Dobj: -1.1313264e-01 
Iter: 10 Ap: 5.91e-01 Pobj: -1.7009929e+01 Ad: 5.91e-01 Dobj: -1.1264932e-01 
Iter: 11 Ap: 6.72